In [18]:
import pandas as pd
import sklearn
import numpy as np

In [17]:
def pre_processing(csv_patch):
  data = pd.read_csv(csv_patch, \
  delimiter = ',', parse_dates=[0], \
  infer_datetime_format=True, na_values='0', \
  header = None)

  data = data.rename(columns={0: 'y'})
  data['y'] = data['y'].astype(str)

  num = 180
  k = 0
  cont = 0
  temp_df = data.head(0)
  while len(data) > 0:
      aux_df = data.head(num)
      temp_df.loc[cont, 'y'] = data.loc[k,'y']

      for x in range(1,53):
        temp_df.loc[cont, x] = aux_df[x].mean()

      data = data.iloc[num:]
      cont += 1
      k += 180

  return temp_df

In [13]:
def concat_date(dft):
  dft[['y', 'm', 'd']] = dft[['y', 'm', 'd']].astype(str)
  dft['start'] = dft['start'].astype(int).astype(str)
  dft['start'] = [('0' + str(x)) if x <10 else str(x) for x in dft['start'].astype(int)]
  dft['m'] = [('0' + str(x)) if x <10 else str(x) for x in dft['m'].astype(int)]
  dft['d'] = [('0' + str(x)) if x <10 else str(x) for x in dft['d'].astype(int)]
  dft["y"]= dft["y"].str.cat(dft['m'], sep ="-")
  dft["y"]= dft["y"].str.cat(dft['d'], sep ="-")
  dft["y"]= dft["y"].str.cat(dft['start'], sep =" ")
  dft['aux'] = pd.Series([':00:00' for x in range(len(dft.index))])
  dft["y"]= dft["y"].str.cat(dft['aux'], sep ="")
  dft = dft.drop(['m', 'd'], axis = 1)
  dft = dft.drop(['aux', 'mid', 'start'], axis=1)

  return dft

In [23]:
data = pre_processing('dsc_fc_summed_spectra_2022_v01.csv')

date = pd.read_csv('date.txt', sep=" ", header=None, names=["y", "m", "d",'start', 'mid', 'k'])
kps = pd.read_csv('justkp.txt', sep=" ", header=None, names=['kp'])
df = date.join(kps['kp'])
df = df.drop('k', axis=1)

df_date = concat_date(df)
final = df_date.merge(data, on='y')

<ipython-input-17-27bd5e9653e0>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[cont, 'y'] = data.loc[k,'y']


In [25]:
df = final
df['temp'] = [1 if x >= 5 else 0 for x in df['kp']]
np_temp = df['temp'].astype(int).to_numpy()
df = df.drop(['kp', 'y'], axis=1)
df = df.fillna(0)
np_values = df.astype(int).to_numpy()

In [26]:
from sklearn.preprocessing import MinMaxScaler
normalizador = MinMaxScaler()
X_norm = normalizador.fit_transform(np_values)

In [44]:
from sklearn.naive_bayes import GaussianNB
X = np.array(X_norm)
Y = np.array(np_temp)
clf = GaussianNB()
clf.fit(X, Y)


test = X_norm[151]
clf.predict([test])

array([0])

In [94]:
#final.to_csv(index=False)

compression_opts = dict(method='zip',
                        archive_name='out.csv')

final.to_csv('out_2019.zip', index=False,
          compression=compression_opts)